In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
pat = pd.read_csv('../../eICU_dbmake/eicu-code/build-db/postgres/csv_files/patient.csv')
admdx = pd.read_csv('../../eICU_dbmake/eicu-code/build-db/postgres/csv_files/admissionDx.csv')
qa = pd.read_csv('../data/intermediate/patid_aki_label_offset.csv')

In [150]:
qa.columns = ['patientunitstayid', 'aki_label', 'aki_offset']
qa.set_index('patientunitstayid', inplace=True)
qa.head()

,aki_label,aki_offset
patientunitstayid,,
141284,0,NaN
141288,0,NaN
141360,0,NaN
141362,0,NaN
141415,0,NaN


In [156]:
qa[qa.aki_offset.notnull()]['aki_offset'].mean()/60

91.28221649484537

In [8]:
print(len(pat))
pat = pat.merge(qa, left_on = 'patientunitstayid', right_on = 'patientunitstayid')
print(len(pat))

200859
90804


In [9]:
print(len(admdx))
admdx = admdx.merge(qa, left_on = 'patientunitstayid', right_on = 'patientunitstayid')
print(len(admdx))

626858
325047


In [80]:
admdx['organ_system'] = admdx['admitdxpath'].str.split(pat="\|Organ System\|", n=1).str[1]
admdx['organ_system'].head(5).tolist()

['Cardiovascular', nan, nan, nan, nan]

In [83]:
admdx['from_OR'] = admdx['admitdxpath'].str.split(pat="O.R. within 4 hours of admission\?\|", n=1).str[1]
admdx['from_OR'].head(5).tolist()

[nan, 'Yes', nan, nan, nan]

In [85]:
admdx['from_OR'].value_counts()

No     71926
Yes    18878
Name: from_OR, dtype: int64

In [100]:
diag_summary = admdx.groupby('patientunitstayid')['organ_system'].apply(lambda x: x.str.cat(sep=' '))
print(diag_summary.value_counts())
print(diag_summary.isnull().sum())

Cardiovascular          38790
Neurologic              17144
Respiratory             14957
Gastrointestinal         9615
Trauma                   4247
Metabolic/Endocrine      3244
Musculoskeletal/Skin     1138
Genitourinary             968
Hematology                585
Transplant                116
Name: organ_system, dtype: int64
0


In [101]:
or_summary = admdx.groupby('patientunitstayid')['from_OR'].apply(lambda x: x.str.cat(sep=' '))
print(or_summary.value_counts())
print(or_summary.isnull().sum())

No     71926
Yes    18878
Name: from_OR, dtype: int64
0


In [103]:
diag_summary_encoded = pd.get_dummies(diag_summary, prefix='organsys', prefix_sep='_', drop_first=False) 
diag_summary_encoded.drop('organsys_Transplant', axis=1, inplace=True)
diag_summary_encoded.head()

,organsys_Cardiovascular,organsys_Gastrointestinal,organsys_Genitourinary,organsys_Hematology,organsys_Metabolic/Endocrine,organsys_Musculoskeletal/Skin,organsys_Neurologic,organsys_Respiratory,organsys_Trauma
patientunitstayid,,,,,,,,,
141284,0,0,0,1,0,0,0,0,0
141288,1,0,0,0,0,0,0,0,0
141360,0,0,0,0,0,0,1,0,0
141362,0,1,0,0,0,0,0,0,0
141415,1,0,0,0,0,0,0,0,0


In [104]:
or_summary_encoded = pd.get_dummies(or_summary, prefix='from_or', prefix_sep='_', drop_first=True) 
or_summary_encoded.head()

,from_or_Yes
patientunitstayid,
141284,0
141288,0
141360,0
141362,1
141415,0


In [111]:
print(pat.gender.value_counts())
print(pat.ethnicity.value_counts())
print(pat.unittype.value_counts())
print(pat.unitdischargestatus.value_counts())

Male       48747
Female     42028
Unknown        8
Other          4
Name: gender, dtype: int64
Caucasian           70459
African American     9436
Other/Unknown        4407
Hispanic             3264
Asian                1612
Native American       563
Name: ethnicity, dtype: int64
Med-Surg ICU    47635
Neuro ICU        8017
MICU             7793
CCU-CTICU        7745
Cardiac ICU      6445
SICU             6438
CSICU            3603
CTICU            3128
Name: unittype, dtype: int64
Alive      86482
Expired     4320
Name: unitdischargestatus, dtype: int64


In [110]:
pat.columns

Index(['patientunitstayid', 'patienthealthsystemstayid', 'gender', 'age',
       'ethnicity', 'hospitalid', 'wardid', 'apacheadmissiondx',
       'admissionheight', 'hospitaladmittime24', 'hospitaladmitoffset',
       'hospitaladmitsource', 'hospitaldischargeyear',
       'hospitaldischargetime24', 'hospitaldischargeoffset',
       'hospitaldischargelocation', 'hospitaldischargestatus', 'unittype',
       'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype',
       'admissionweight', 'dischargeweight', 'unitdischargetime24',
       'unitdischargeoffset', 'unitdischargelocation', 'unitdischargestatus',
       'uniquepid', 'aki_label', 'aki_offset'],
      dtype='object')

In [112]:
pat[['patientunitstayid', 'gender', 'age', 'ethnicity', 'unittype', 'unitvisitnumber', 'unitdischargeoffset', \
     'unitdischargestatus']].isnull().sum()

patientunitstayid         0
gender                   17
age                       0
ethnicity              1063
unittype                  0
unitvisitnumber           0
unitdischargeoffset       0
unitdischargestatus       2
dtype: int64

In [121]:
pat.loc[pat.age == '> 89', 'age'] = 91
pat.age.mode()

0    91
dtype: object

In [126]:
pat = pat.apply(lambda x:x.fillna(x.value_counts().index[0]))
pat[['patientunitstayid', 'gender', 'age', 'ethnicity', 'unittype', 'unitvisitnumber', 'unitdischargeoffset', \
     'unitdischargestatus']].isnull().sum()

patientunitstayid      0
gender                 0
age                    0
ethnicity              0
unittype               0
unitvisitnumber        0
unitdischargeoffset    0
unitdischargestatus    0
dtype: int64

In [128]:
pat.loc[pat.gender=='Other', 'gender']='Male'
pat.loc[pat.gender=='Unknown', 'gender']='Male'
print(pat.gender.value_counts())

Male      48776
Female    42028
Name: gender, dtype: int64


In [145]:
pat.set_index('patientunitstayid', inplace=True)
pat.head()

,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,hospitaladmitoffset,...,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid,aki_label,aki_offset
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141284,129017,Male,63,Caucasian,73,92,Anemia,188.0,23:10:00,-15,...,admit,68.0,88.5,10:01:00,2076,Floor,Alive,002-72371,0,1589.0
141288,129020,Female,61,Caucasian,67,109,"Sepsis, pulmonary",162.6,17:32:00,-171,...,admit,68.0,92.2,23:34:00,1631,Floor,Alive,002-67735,0,1630.0
141360,129080,Male,48,Caucasian,73,102,"CVA, cerebrovascular accident/stroke",175.3,18:39:00,-30,...,admit,68.0,92.3,22:58:00,3109,Floor,Alive,002-34250,0,1702.0
141362,129082,Male,65,Caucasian,73,93,"Cholecystectomy/cholangitis, surgery for (gall...",172.7,21:03:00,-1505,...,admit,68.0,65.1,02:06:00,3118,Floor,Alive,002-66659,0,1708.0
141415,129126,Male,30,Caucasian,66,90,"Thrombosis, vascular (deep vein)",193.0,17:04:00,-347,...,admit,87.1,86.7,00:41:00,4430,Floor,Alive,002-42357,0,1740.0


In [146]:
#p.patientUnitStayID, p.gender, p.age, p.ethnicity, p.unitType, p.unitVisitNumber, p.unitDischargeOffset, p.unitDischargeStatus
cat_columns = ['gender', 'ethnicity', 'unittype', 'unitdischargestatus']
pat_cat_encoded = pd.get_dummies(pat[cat_columns], prefix_sep='_', drop_first=False)
pat_cat_encoded.head()

,gender_Female,gender_Male,ethnicity_African American,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,unittype_CCU-CTICU,unittype_CSICU,unittype_CTICU,unittype_Cardiac ICU,unittype_MICU,unittype_Med-Surg ICU,unittype_Neuro ICU,unittype_SICU,unitdischargestatus_Alive,unitdischargestatus_Expired
patientunitstayid,,,,,,,,,,,,,,,,,,
141284,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0
141288,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
141360,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0
141362,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
141415,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0


In [147]:
cols_to_drop = ['gender_Male', 'ethnicity_Native American', 'unittype_CTICU', 'unitdischargestatus_Expired']
pat_cat_encoded.drop(columns=cols_to_drop, inplace=True)
pat_cat_encoded.columns.tolist()

['gender_Female',
 'ethnicity_African American',
 'ethnicity_Asian',
 'ethnicity_Caucasian',
 'ethnicity_Hispanic',
 'ethnicity_Other/Unknown',
 'unittype_CCU-CTICU',
 'unittype_CSICU',
 'unittype_Cardiac ICU',
 'unittype_MICU',
 'unittype_Med-Surg ICU',
 'unittype_Neuro ICU',
 'unittype_SICU',
 'unitdischargestatus_Alive']

In [130]:
# keep track of droped columns for interpretation
droped_cols = ['organsys_Transplant', 'ethnicity_Native American', 'unittype_CTICU']

In [187]:
# put together all relevant encoded values
# qa
# diag_summary_encoded
# or_summary_encoded
# pat_cat_encoded
# pat[['patientunitstayid', 'age', 'unitvisitnumber', 'unitdischargeoffset']]
#pat.age = pat.age.astype(int) # age is stored as string, convert to int
combined_df = pd.concat([qa, diag_summary_encoded, or_summary_encoded, pat_cat_encoded, \
                         pat[['age', 'unitvisitnumber', 'unitdischargeoffset']]], \
                        axis=1, ignore_index=False)
print(len(combined_df))
print(combined_df.isnull().sum())

90804
aki_label                            0
aki_offset                       88476
organsys_Cardiovascular              0
organsys_Gastrointestinal            0
organsys_Genitourinary               0
organsys_Hematology                  0
organsys_Metabolic/Endocrine         0
organsys_Musculoskeletal/Skin        0
organsys_Neurologic                  0
organsys_Respiratory                 0
organsys_Trauma                      0
from_or_Yes                          0
gender_Female                        0
ethnicity_African American           0
ethnicity_Asian                      0
ethnicity_Caucasian                  0
ethnicity_Hispanic                   0
ethnicity_Other/Unknown              0
unittype_CCU-CTICU                   0
unittype_CSICU                       0
unittype_Cardiac ICU                 0
unittype_MICU                        0
unittype_Med-Surg ICU                0
unittype_Neuro ICU                   0
unittype_SICU                        0
unitdischargestatus

In [161]:
combined_df.to_csv('../data/intermediate/pat_admit_diag_cleaned.csv')

In [253]:
# keep track of index columns
index_cols = ['patientunitstayid'] # currently used as index

# keep track of cols important for stratification
strat_cols = ['unitvisitnumber', 'aki_label', 'unitdischargestatus_Alive']

# keep track of columns important for time tracking
time_cols = ['aki_offset', 'unitdischargeoffset']

# keep track of numerical columns
num_cols = ['age']

# keep track of categorical columns
cat_cols = ['organsys_Cardiovascular', 'organsys_Gastrointestinal',
       'organsys_Genitourinary', 'organsys_Hematology',
       'organsys_Metabolic/Endocrine', 'organsys_Musculoskeletal/Skin',
       'organsys_Neurologic', 'organsys_Respiratory', 'organsys_Trauma',
       'from_or_Yes', 'gender_Female', 'ethnicity_African American',
       'ethnicity_Asian', 'ethnicity_Caucasian', 'ethnicity_Hispanic',
       'ethnicity_Other/Unknown', 'unittype_CCU-CTICU', 'unittype_CSICU',
       'unittype_Cardiac ICU', 'unittype_MICU', 'unittype_Med-Surg ICU',
       'unittype_Neuro ICU', 'unittype_SICU', 'unitdischargestatus_Alive']

# keep track of target variables
target_cols = ['aki_label', 'unitdischargestatus_Alive']

In [162]:
combined_df.columns

Index(['aki_label', 'aki_offset', 'organsys_Cardiovascular',
       'organsys_Gastrointestinal', 'organsys_Genitourinary',
       'organsys_Hematology', 'organsys_Metabolic/Endocrine',
       'organsys_Musculoskeletal/Skin', 'organsys_Neurologic',
       'organsys_Respiratory', 'organsys_Trauma', 'from_or_Yes',
       'gender_Female', 'ethnicity_African American', 'ethnicity_Asian',
       'ethnicity_Caucasian', 'ethnicity_Hispanic', 'ethnicity_Other/Unknown',
       'unittype_CCU-CTICU', 'unittype_CSICU', 'unittype_Cardiac ICU',
       'unittype_MICU', 'unittype_Med-Surg ICU', 'unittype_Neuro ICU',
       'unittype_SICU', 'unitdischargestatus_Alive', 'age', 'unitvisitnumber',
       'unitdischargeoffset'],
      dtype='object')

In [164]:
lab = pd.read_csv('../data/intermediate/lab_summary.csv', header=None)
lab.head()

,0,1,2,3,4
0,141284,BUN,20.0,20.0,0.0
1,141284,Hct,27.1,28.4,1.3
2,141284,Hgb,8.8,9.3,0.5
3,141284,MCH,26.3,26.3,0.0
4,141284,MCHC,32.7,32.7,0.0


In [165]:
lab.columns = ['patientunitstayid', 'labname', 'min_result', 'max_result', 'delta_result']
lab.set_index(['patientunitstayid', 'labname'], inplace=True)
lab.head()

min_result  max_result  delta_result
patientunitstayid labname                                      
141284            BUN            20.0        20.0           0.0
                  Hct            27.1        28.4           1.3
                  Hgb             8.8         9.3           0.5
                  MCH            26.3        26.3           0.0
                  MCHC           32.7        32.7           0.0

In [166]:
lab = lab.unstack(level=1)
lab.head()

min_result                                                 \
labname                  BUN   FiO2  HCO3   Hct   Hgb   MCH  MCHC   MCV MPV   
patientunitstayid                                                             
141284                  20.0    NaN   NaN  27.1   8.8  26.3  32.7  80.2 NaN   
141288                   NaN  100.0  24.0  35.4  11.8  32.6  33.3  97.8 NaN   
141360                   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN NaN   
141362                   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN NaN   
141415                  13.0    NaN   NaN  37.4  12.2  32.0  32.1  99.5 NaN   

                         ... delta_result                                    \
labname             RBC  ...     chloride creatinine glucose magnesium   pH   
patientunitstayid        ...                                                  
141284             3.54  ...          0.0       0.00     0.0       NaN  NaN   
141288             3.62  ...          NaN        NaN     NaN       NaN  0.0   
141360              NaN  ...          NaN        NaN     NaN       NaN  NaN   
141362              NaN  ...          NaN        NaN     NaN       NaN  NaN   
141415             3.76  ...          0.0       0.15    10.0       NaN  NaN   

                                                                
labname           paCO2 paO2 platelets x 1000 potassium sodium  
patientunitstayid                                               
141284              NaN  NaN              0.0       0.0    0.0  
141288              0.0  0.0              0.0       NaN    NaN  
141360              NaN  NaN              NaN       NaN    NaN  
141362              NaN  NaN              NaN       NaN    NaN  
141415              NaN  NaN             21.0       0.2    1.0  

[5 rows x 78 columns]

In [167]:
lab.columns = lab.columns.map('_'.join)
lab.head()

,min_result_BUN,min_result_FiO2,min_result_HCO3,min_result_Hct,min_result_Hgb,min_result_MCH,min_result_MCHC,min_result_MCV,min_result_MPV,min_result_RBC,...,delta_result_chloride,delta_result_creatinine,delta_result_glucose,delta_result_magnesium,delta_result_pH,delta_result_paCO2,delta_result_paO2,delta_result_platelets x 1000,delta_result_potassium,delta_result_sodium
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141284,20.0,NaN,NaN,27.1,8.8,26.3,32.7,80.2,NaN,3.54,...,0.0,0.00,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
141288,NaN,100.0,24.0,35.4,11.8,32.6,33.3,97.8,NaN,3.62,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN
141360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141415,13.0,NaN,NaN,37.4,12.2,32.0,32.1,99.5,NaN,3.76,...,0.0,0.15,10.0,NaN,NaN,NaN,NaN,21.0,0.2,1.0


In [174]:
lab.columns.tolist()

['min_result_BUN',
 'min_result_FiO2',
 'min_result_HCO3',
 'min_result_Hct',
 'min_result_Hgb',
 'min_result_MCH',
 'min_result_MCHC',
 'min_result_MCV',
 'min_result_MPV',
 'min_result_RBC',
 'min_result_RDW',
 'min_result_WBC x 1000',
 'min_result_anion gap',
 'min_result_bedside glucose',
 'min_result_bicarbonate',
 'min_result_calcium',
 'min_result_chloride',
 'min_result_creatinine',
 'min_result_glucose',
 'min_result_magnesium',
 'min_result_pH',
 'min_result_paCO2',
 'min_result_paO2',
 'min_result_platelets x 1000',
 'min_result_potassium',
 'min_result_sodium',
 'max_result_BUN',
 'max_result_FiO2',
 'max_result_HCO3',
 'max_result_Hct',
 'max_result_Hgb',
 'max_result_MCH',
 'max_result_MCHC',
 'max_result_MCV',
 'max_result_MPV',
 'max_result_RBC',
 'max_result_RDW',
 'max_result_WBC x 1000',
 'max_result_anion gap',
 'max_result_bedside glucose',
 'max_result_bicarbonate',
 'max_result_calcium',
 'max_result_chloride',
 'max_result_creatinine',
 'max_result_glucose',
 'm

In [188]:
combined_df = combined_df.merge(lab, left_index=True, right_index=True)
missing_vals = combined_df.isnull().sum()
missing_vals_fract = missing_vals/len(combined_df)
len(missing_vals_fract[missing_vals_fract>0.2])

79

In [189]:
len(missing_vals_fract[missing_vals_fract>0.3])

55

In [192]:
missing_vals_fract[(missing_vals_fract<0.3) & (missing_vals_fract>0)].index.tolist()

['min_result_BUN',
 'min_result_Hct',
 'min_result_Hgb',
 'min_result_chloride',
 'min_result_creatinine',
 'min_result_glucose',
 'min_result_potassium',
 'min_result_sodium',
 'max_result_BUN',
 'max_result_Hct',
 'max_result_Hgb',
 'max_result_chloride',
 'max_result_creatinine',
 'max_result_glucose',
 'max_result_potassium',
 'max_result_sodium',
 'delta_result_BUN',
 'delta_result_Hct',
 'delta_result_Hgb',
 'delta_result_chloride',
 'delta_result_creatinine',
 'delta_result_glucose',
 'delta_result_potassium',
 'delta_result_sodium']

In [200]:
# really care about the positive class
pos_miss_vals = combined_df.loc[combined_df['aki_label']==1].isnull().sum()
pos_miss_vals[(pos_miss_vals < 446) & (pos_miss_vals>0)]

min_result_potassium      359
min_result_sodium         427
max_result_potassium      359
max_result_sodium         427
delta_result_potassium    359
delta_result_sodium       427
dtype: int64

In [196]:
combined_df.aki_label.sum()

2229

In [203]:
combined_df = pd.read_csv('../data/intermediate/pat_admit_diag_cleaned.csv')
combined_df.set_index('patientunitstayid', inplace=True)
combined_df.head()

,aki_label,aki_offset,organsys_Cardiovascular,organsys_Gastrointestinal,organsys_Genitourinary,organsys_Hematology,organsys_Metabolic/Endocrine,organsys_Musculoskeletal/Skin,organsys_Neurologic,organsys_Respiratory,...,unittype_CSICU,unittype_Cardiac ICU,unittype_MICU,unittype_Med-Surg ICU,unittype_Neuro ICU,unittype_SICU,unitdischargestatus_Alive,age,unitvisitnumber,unitdischargeoffset
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141284,0,NaN,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,63,1,2076
141288,0,NaN,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,61,1,1631
141360,0,NaN,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,48,1,3109
141362,0,NaN,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,65,1,3118
141415,0,NaN,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,30,1,4430


In [204]:
lab = pd.read_csv('../data/intermediate/lab_summary_prior.csv', header=None)
lab.columns = ['patientunitstayid', 'labname', 'min_result', 'max_result', 'delta_result']
lab.set_index(['patientunitstayid', 'labname'], inplace=True)
lab = lab.unstack(level=1)
lab.columns = lab.columns.map('_'.join)
lab.head()


,min_result_BUN,min_result_FiO2,min_result_HCO3,min_result_Hct,min_result_Hgb,min_result_MCH,min_result_MCHC,min_result_MCV,min_result_MPV,min_result_RBC,...,delta_result_chloride,delta_result_creatinine,delta_result_glucose,delta_result_magnesium,delta_result_pH,delta_result_paCO2,delta_result_paO2,delta_result_platelets x 1000,delta_result_potassium,delta_result_sodium
patientunitstayid,,,,,,,,,,,,,,,,,,,,,
141284,20.0,NaN,NaN,27.1,8.8,26.3,32.7,80.2,NaN,3.54,...,0.0,0.00,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
141288,17.0,100.0,24.0,35.4,11.8,31.8,32.5,97.8,NaN,3.62,...,0.0,0.00,0.0,NaN,0.0,0.0,0.0,31.0,0.0,0.0
141360,13.0,NaN,NaN,48.9,16.6,29.9,33.9,88.1,NaN,5.55,...,0.0,0.00,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
141362,16.0,NaN,NaN,36.6,11.8,29.8,32.2,92.2,NaN,3.87,...,4.0,0.21,20.0,0.5,NaN,NaN,NaN,13.0,0.8,2.0
141415,13.0,NaN,NaN,37.4,12.2,32.0,32.1,96.8,NaN,3.76,...,3.0,0.29,51.0,NaN,NaN,NaN,NaN,58.0,0.4,2.0


In [205]:
combined_df = combined_df.merge(lab, left_index=True, right_index=True)
missing_vals = combined_df.isnull().sum()
missing_vals_fract = missing_vals/len(combined_df)
len(missing_vals_fract[missing_vals_fract>0.2])

28

In [206]:
len(missing_vals_fract[missing_vals_fract>0.1])

46

In [233]:
pos_miss_vals = combined_df.loc[combined_df['aki_label']==1].isnull().sum()
lab_cols_todrop = pos_miss_vals[pos_miss_vals > 200].index.tolist()
lab_cols_todrop

['min_result_FiO2',
 'min_result_HCO3',
 'min_result_MCH',
 'min_result_MCHC',
 'min_result_MCV',
 'min_result_MPV',
 'min_result_RDW',
 'min_result_anion gap',
 'min_result_bedside glucose',
 'min_result_magnesium',
 'min_result_pH',
 'min_result_paCO2',
 'min_result_paO2',
 'max_result_FiO2',
 'max_result_HCO3',
 'max_result_MCH',
 'max_result_MCHC',
 'max_result_MCV',
 'max_result_MPV',
 'max_result_RDW',
 'max_result_anion gap',
 'max_result_bedside glucose',
 'max_result_magnesium',
 'max_result_pH',
 'max_result_paCO2',
 'max_result_paO2',
 'delta_result_FiO2',
 'delta_result_HCO3',
 'delta_result_MCH',
 'delta_result_MCHC',
 'delta_result_MCV',
 'delta_result_MPV',
 'delta_result_RDW',
 'delta_result_anion gap',
 'delta_result_bedside glucose',
 'delta_result_magnesium',
 'delta_result_pH',
 'delta_result_paCO2',
 'delta_result_paO2']

In [234]:
combined_df_filtered = combined_df.drop(columns = lab_cols_todrop)

In [247]:
row_null_sum = combined_df_filtered.loc[combined_df['aki_label']==1].isnull().sum(axis=1).sort_values(ascending=True)
print(len(row_null_sum))
len(row_null_sum[row_null_sum<10])

2307


2116

In [248]:
combined_df_filtered_dr = combined_df_filtered[combined_df_filtered.isnull().sum(axis=1)<10]
print(len(combined_df_filtered_dr[combined_df_filtered_dr['aki_label']==1]))
print(len(combined_df_filtered_dr[combined_df_filtered_dr['aki_label']==0]))

2116
76196


In [249]:
combined_df_filtered_dr.isnull().sum()

aki_label                            0
aki_offset                       76196
organsys_Cardiovascular              0
organsys_Gastrointestinal            0
organsys_Genitourinary               0
                                 ...  
delta_result_creatinine            132
delta_result_glucose               444
delta_result_platelets x 1000      735
delta_result_potassium              72
delta_result_sodium                  5
Length: 68, dtype: int64

In [254]:
# Note I will impute missing values after splitting into train test
# strategies median, or sklearn Multivariate feature imputation

# keep track of new numerical features
num_cols += ['min_result_BUN', 'min_result_Hct',
       'min_result_Hgb', 'min_result_RBC', 'min_result_WBC x 1000',
       'min_result_bicarbonate', 'min_result_calcium', 'min_result_chloride',
       'min_result_creatinine', 'min_result_glucose',
       'min_result_platelets x 1000', 'min_result_potassium',
       'min_result_sodium', 'max_result_BUN', 'max_result_Hct',
       'max_result_Hgb', 'max_result_RBC', 'max_result_WBC x 1000',
       'max_result_bicarbonate', 'max_result_calcium', 'max_result_chloride',
       'max_result_creatinine', 'max_result_glucose',
       'max_result_platelets x 1000', 'max_result_potassium',
       'max_result_sodium', 'delta_result_BUN', 'delta_result_Hct',
       'delta_result_Hgb', 'delta_result_RBC', 'delta_result_WBC x 1000',
       'delta_result_bicarbonate', 'delta_result_calcium',
       'delta_result_chloride', 'delta_result_creatinine',
       'delta_result_glucose', 'delta_result_platelets x 1000',
       'delta_result_potassium', 'delta_result_sodium']

In [251]:
combined_df_filtered_dr.columns

Index(['aki_label', 'aki_offset', 'organsys_Cardiovascular',
       'organsys_Gastrointestinal', 'organsys_Genitourinary',
       'organsys_Hematology', 'organsys_Metabolic/Endocrine',
       'organsys_Musculoskeletal/Skin', 'organsys_Neurologic',
       'organsys_Respiratory', 'organsys_Trauma', 'from_or_Yes',
       'gender_Female', 'ethnicity_African American', 'ethnicity_Asian',
       'ethnicity_Caucasian', 'ethnicity_Hispanic', 'ethnicity_Other/Unknown',
       'unittype_CCU-CTICU', 'unittype_CSICU', 'unittype_Cardiac ICU',
       'unittype_MICU', 'unittype_Med-Surg ICU', 'unittype_Neuro ICU',
       'unittype_SICU', 'unitdischargestatus_Alive', 'age', 'unitvisitnumber',
       'unitdischargeoffset', 'min_result_BUN', 'min_result_Hct',
       'min_result_Hgb', 'min_result_RBC', 'min_result_WBC x 1000',
       'min_result_bicarbonate', 'min_result_calcium', 'min_result_chloride',
       'min_result_creatinine', 'min_result_glucose',
       'min_result_platelets x 1000', 'min_resul

In [257]:
vitals = pd.read_csv('../data/intermediate/vitalperiodic_summary.csv', header=None)
vitals.columns = ['patientunitstayid','min_sao2', 'max_sao2', 'mean_sao2', 'min_heartrate', 'max_heartrate', \
                  'mean_heartrate', 'min_respiration', 'max_respiration', 'mean_respiration']
vitals.set_index('patientunitstayid', inplace=True)
vitals.head()

,min_sao2,max_sao2,mean_sao2,min_heartrate,max_heartrate,mean_heartrate,min_respiration,max_respiration,mean_respiration
patientunitstayid,,,,,,,,,
141284,92.0,100.0,96.073529,70.0,106.0,84.752613,10.0,36.0,21.254355
141288,90.0,100.0,95.219512,80.0,106.0,91.857143,11.0,46.0,21.972125
141360,96.0,100.0,98.602230,76.0,120.0,89.808118,11.0,36.0,17.656827
141362,80.0,99.0,93.338129,92.0,120.0,103.592982,11.0,42.0,24.518182
141415,97.0,100.0,98.353488,61.0,99.0,72.034722,8.0,57.0,15.880000


In [260]:
combined_df_filtered_dr_vit = combined_df_filtered_dr.merge(vitals, left_index=True, right_index=True)
row_null_sum = combined_df_filtered_dr_vit.loc[combined_df_filtered_dr_vit['aki_label']==1][vitals.columns].isnull().sum().sort_values(ascending=True)
row_null_sum

min_heartrate         0
max_heartrate         0
mean_heartrate        0
min_sao2             10
max_sao2             10
mean_sao2            10
min_respiration     140
max_respiration     140
mean_respiration    140
dtype: int64

In [261]:
row_null_sum = combined_df_filtered_dr_vit.loc[combined_df_filtered_dr_vit['aki_label']==0][vitals.columns].isnull().sum().sort_values(ascending=True)
row_null_sum

min_heartrate        162
max_heartrate        162
mean_heartrate       162
min_sao2             733
max_sao2             733
mean_sao2            733
min_respiration     5388
max_respiration     5388
mean_respiration    5388
dtype: int64

In [264]:
# respiration columns will be droped, for the rest drop the rows that are null
combined_df_filtered_dr_vit.drop(columns=['min_respiration', 'max_respiration', 'mean_respiration'])
combined_df_filtered_dr_vit_final = combined_df_filtered_dr_vit[combined_df_filtered_dr_vit[['min_heartrate', 'max_heartrate', \
                                                                                     'mean_heartrate', 'min_sao2', 'max_sao2', \
                                                                                     'mean_sao2']].isnull().sum(axis=1)==0]
print(combined_df_filtered_dr_vit_final.aki_label.sum())
print(len(combined_df_filtered_dr_vit_final))

2099
76624


In [266]:
combined_df_filtered_dr_vit_final.to_csv('../data/final/vitalperiodic_summary.csv')